## Imports

## Data Loading & Pre-Processing
-

## Dimension Reduction Using SVD
-

## Model Training - Logistic Regression With SVD

### Model Visualization Confusion Matrix

##### How does reducing dimensions affect model accuracy and speed?
-